# Práctica de Laboratorio en NLP - Tópicos
Tema: Extracción de tópicos

Introducción:

En esta práctica, trabajaremos en un proyecto de análisis de reclamaciones de un restaurante. La empresa desea saber de qué se quejan más los comensales, para identificar áreas de mejora. 


In [ ]:
### Importación de librerías

import spacy
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nlp = spacy.load("es_core_news_lg")

In [ ]:
sentencias = [
    "La comida llegó fría y tardó mucho en ser servida.",
    "El servicio fue lento y el personal parecía desinteresado.",
    "Pedimos un plato vegetariano y nos trajeron uno con carne.",
    "Las mesas estaban sucias y no habían sido limpiadas correctamente.",
    "No había suficiente iluminación en el área de comedor.",
    "El restaurante estaba abarrotado y no se respetaron las reservas.",
    "La música estaba demasiado alta y no se podía mantener una conversación.",
    "La bebida que pedimos nunca llegó a la mesa.",
    "Los platos estaban mal sazonados y carecían de sabor.",
    "El menú tenía errores de ortografía y gramática.",
    "El restaurante tenía un olor desagradable.",
    "No había opciones vegetarianas en el menú.",
    "Se nos cobró de más en la factura.",
    "El baño del restaurante estaba sucio y sin papel higiénico.",
    "El vino que pedimos estaba agrio y parecía estar mal almacenado.",
    "La carne de mi plato estaba cruda en el centro.",
    "Las sillas eran incómodas y difíciles de mover.",
    "El personal fue grosero y poco atento.",
    "El restaurante estaba demasiado caliente y sin aire acondicionado.",
    "No se nos proporcionaron utensilios de mesa ni servilletas.",
    "El postre que pedimos estaba rancio y no era comestible.",
    "El menú era limitado y no ofrecía opciones para alergias alimentarias.",
    "El restaurante tenía una plaga de insectos.",
    "Los platos se veían descuidados y mal presentados.",
    "La factura tenía errores en los cálculos de los precios."
]




In [ ]:
import re
import nltk
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('spanish')


def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

norm_corpus=[]

for document in sentencias:
    norm_corpus.append(normalize_document(document))

norm_corpus


In [ ]:
### Creación del vectorizador y generación de la matriz Tf-idf

tfifd_vec = TfidfVectorizer()

TFIDF = tfifd_vec.fit_transform(norm_corpus)

TFIDF

In [ ]:
# Visualización de la matriz

# Obtenemos el vocabulario para poner las etiquetas de las columnaas
vocab = tfifd_vec.get_feature_names()

print("Palabras en el vocabulario: ", len(vocab))
print(vocab)

In [ ]:
# Y construimos un dataframe para mostrar el resultado: por cada documento las ocurrencias de cada token
pd.DataFrame(TFIDF.toarray(), columns=vocab)

## Implementación de LDA

In [ ]:
# Creación de un objeto de clase LDA con sus componentes
lda_model = LatentDirichletAllocation(n_components = 3, max_iter = 20, random_state = 20)

# Extracción de los tópicos
X_topics = lda_model.fit_transform(TFIDF)

topic_words = lda_model.components_
topic_words

## Palabras en cada tópico

In [ ]:
#  Número de palabras a extraer de cada tópico

n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    sorted_topic_dist = np.argsort(topic_dist)
    
    topic_words = np.array(vocab)[sorted_topic_dist]
    
    topic_words = topic_words[:-n_top_words:-1]
    print ("Tópico", str(i+1), topic_words)

## Distribución de documentos por tópico

In [ ]:
doc_topic = lda_model.transform(TFIDF)  

for n in range(doc_topic.shape[0]):
    topic_doc = doc_topic[n].argmax()
    print ("Documento", n+1, " -- Tópico:" ,topic_doc)

# Cómo visualizar los tópicos en el espacio

In [ ]:
!pip install pyLDAvis

In [ ]:
# Importa las bibliotecas necesarias
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import numpy as np


# Prepara los datos para PyLDAvis
panel = pyLDAvis.sklearn.prepare(lda_model, TFIDF, TFIDFVectorizer, mds='tsne')

# Visualiza el modelo LDA
pyLDAvis.display(panel)
